In [2]:
#load data

import os                   
import skimage              
import numpy as np           
from skimage import data,transform
from skimage import img_as_float
from skimage.color import rgb2gray     
import matplotlib.pyplot as plt
import random   
from PIL import Image


def load_data(data_directory):
    directories=[d for d in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory,d))]
    #d is every classification file
    labels=[]
    images=[]
    names=[]
    for d in directories:
        #get path of each class
        label_directory=os.path.join(data_directory,d)
        file_names=[os.path.join(label_directory,f) for f in os.listdir(label_directory) if f.endswith(".jpeg") or f.endswith('.JPG') ]
        for f in file_names:

            img=skimage.data.imread(f, as_grey='True')
            images.append(img)              #read image
            labels.append(d)                   #read label
            names.append(f)
    return images,labels,names

#images and labels are list

ROOT_PATH="/Users/chenlingna/Desktop"
color = 150/255
data_directory=os.path.join(ROOT_PATH,"Melbourne_Classification")



In [3]:
#Load Data Set
images_Total,labels_Total,names_Total=load_data(data_directory)


In [4]:
#Split the dataset into training and testing
from sklearn.model_selection import train_test_split
images, test_images, labels, test_labels, names, test_names  = train_test_split(images_Total, labels_Total, names_Total, test_size=0.3, random_state=90051)

In [5]:
#Manage Training Set
# images,labels,names=load_data(train_data_directory)

images28=[]
for img in images:

    images28.append(img)
    
# Rescale the images in the `images` array
images28 = [transform.resize(image, (28, 28)) for image in images28]
# Convert `images28` to an array
images28 = np.array(images28)
# # Convert `images` to grayscale
# images28 = rgb2gray(images28)





/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [6]:
#Load Testing Set
# test_images, test_labels, test_names = load_data(test_data_directory)

test_images28=[]
for img in test_images:
#     img[img>color]=1
#     img[img<color]=0
    test_images28.append(img)

# Transform the images to 28 by 28 pixels
test_images28 = [transform.resize(image, (28, 28)) for image in test_images28]
test_images28 = np.array(test_images28)
# # Convert to grayscale
# from skimage.color import rgb2gray

# test_images28 = rgb2gray(np.array(test_images28))


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [7]:
class DatasetIterator:
    """
    An iterator that returns randomized batches from a data set (with features and labels)
    """
    def __init__(self, features, labels, batch_size):
        assert(features.shape[0]==labels.shape[0])
        assert(batch_size > 0 and batch_size <= features.shape[0])
        self.features = features
        self.labels = labels
        self.num_instances = features.shape[0]
        self.batch_size = batch_size
        self.num_batches = self.num_instances//self.batch_size
        if (self.num_instances%self.batch_size!=0):
            self.num_batches += 1
        self._i = 0
        self._rand_ids = None

    def __iter__(self):
        self._i = 0
        self._rand_ids = np.random.permutation(self.num_instances)
        return self
    
    def next(self):
        self.__next__(self)
    
    def __next__(self):
        if self.num_instances - self._i >= self.batch_size:
            this_rand_ids = self._rand_ids[self._i:self._i + self.batch_size]
            self._i += self.batch_size
            return self.features[this_rand_ids], self.labels[this_rand_ids]
        elif self.num_instances - self._i > 0:
            this_rand_ids = self._rand_ids[self._i::]
            self._i = self.num_instances
            return self.features[this_rand_ids], self.labels[this_rand_ids]
        else:
            raise StopIteration()
            
batch_size = 100
labels=np.array(labels)
train_iterator = DatasetIterator(images28, labels, batch_size)

In [8]:
#CNN
import tensorflow as tf
IM_WIDTH = images28.shape[1]      # width of an image in pixels
IM_HEIGHT = images28.shape[2] 
NUM_CLASSES = 2
tf.reset_default_graph()
with tf.variable_scope('input'):
    X = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28], name='image')
    Y = tf.placeholder(tf.int32, [None], name='label')
    
DEPTH_C1 = 8       # depth of convolutional layer #1
DEPTH_C2 = 16      # depth of convolutional layer #2
UNITS_D1 = 256     # number of neurons in dense layer #1
with tf.variable_scope('cnn_model'):
    # Boolean placeholder which is set to True for training, and False for inference.
    # This is required to implement dropout. 
    training_mode = tf.placeholder(dtype=tf.bool, name='training_mode')
    
    # Input Layer
    input_layer = tf.reshape(X, [-1, IM_WIDTH, IM_HEIGHT, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=DEPTH_C1, kernel_size=[5, 5], 
                             padding='same', activation=tf.nn.relu, use_bias=True, 
                             name='conv_layer_1')

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, 
                                    name='pool_layer_1')

    # Convolutional Layer #2 
    conv2 = tf.layers.conv2d(inputs=pool1, filters=DEPTH_C2, kernel_size=[5, 5], 
                             padding='same', activation=tf.nn.relu, use_bias=True, 
                             name='conv_layer_2') # fill in

    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, 
                                    name='pool_layer_2') # fill in

    # Dense Layer #1
    pool2_flat = tf.reshape(pool2, shape=[-1, 7*7*DEPTH_C2], name='pool_layer_2_flat')
    dense = tf.layers.dense(inputs=pool2_flat, units=UNITS_D1, activation=tf.nn.relu, 
                            name='dense_layer_1')
    dropout = tf.layers.dropout(inputs=dense, rate=0.7, training=training_mode, name='dropout')

    # Dense Layer #2 (Logits Layer)
    logits = tf.layers.dense(inputs=dropout, units=NUM_CLASSES, use_bias=True,
                             name='dense_layer_2')
    
    # Predicted labels
    predictions = tf.argmax(logits, axis=1)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
with tf.variable_scope('loss'):
    loss = tf.losses.sparse_softmax_cross_entropy(labels=Y, logits=logits)
with tf.variable_scope('train'):
    opt = tf.train.AdamOptimizer(learning_rate=0.001)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = opt.minimize(loss=loss, global_step=global_step)

In [10]:
with tf.variable_scope('evaluation'):
    acc, acc_op = tf.metrics.accuracy(labels=Y, predictions=predictions, name='accuracy')
    loss_summary = tf.summary.scalar('loss', loss)
    acc_summary = tf.summary.scalar('accuracy', acc)
    eval_summaries = tf.summary.merge([loss_summary, acc_summary])

In [11]:
with tf.variable_scope('cnn_model/conv_layer_1', reuse=True):
    kernel = tf.get_variable('kernel')
    with tf.variable_scope('visualization'):
        # scale weights to [0 1]
        x_min = tf.reduce_min(kernel)
        x_max = tf.reduce_max(kernel)
        kernel_0_to_1 = (kernel - x_min) / (x_max - x_min)

        # to tf.summary.image format
        kernel_transposed = tf.transpose(kernel_0_to_1, [3, 0, 1, 2])

        # this will display 5 filters from the 8 in conv_layer_1
        filter_summary = tf.summary.image('filters', kernel_transposed, max_outputs=5)

In [12]:
LOG_DIR = os.path.join(os.curdir, 'parkingSign_log')
NUM_EPOCHS = 400
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
# with tf.Session() as sess:
sess=tf.Session()
sess.run(init)
# Instantiate writers for TensorBoard (for saving serialized summaries to disk)
train_summary_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'), sess.graph)
test_summary_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'), sess.graph)
# Run optimizer for multiple epochs
for epoch in range(NUM_EPOCHS):
    print("Starting epoch {}.".format(epoch))
    for X_batch, Y_batch in train_iterator:
        # Run a training step
        _, step = sess.run([train_op, global_step],
                           feed_dict={X: X_batch, Y: Y_batch, training_mode: True})
        # Every 100 batches compute the accuracy on the training set and save the filters in the first convolutional layer
        if (step % 100 == 0 and step > 0):
            train_accuracy, eval_s, filter_s = sess.run([acc_op, eval_summaries, filter_summary], 
                              feed_dict={X: images28, Y: labels, training_mode: False})
            train_summary_writer.add_summary(eval_s, global_step=step)
            train_summary_writer.add_summary(filter_s, global_step=step)
            print("\tTraining accuracy at step {}: {}.".format(step, train_accuracy))
        # Every 10 batches compute the accuracy on the test set.
        if (step % 10 == 0):
            test_accuracy, eval_s = sess.run([acc_op, eval_summaries], 
                             feed_dict={X: test_images28, Y: test_labels, training_mode: False})
            test_summary_writer.add_summary(eval_s, global_step=step)
print("Optimization complete.")
train_summary_writer.close()
test_summary_writer.close()

Starting epoch 0.
Starting epoch 1.
	Training accuracy at step 100: 0.9558823704719543.
Starting epoch 2.
	Training accuracy at step 200: 0.9559026956558228.
Starting epoch 3.
	Training accuracy at step 300: 0.9559090733528137.
Starting epoch 4.
Starting epoch 5.
	Training accuracy at step 400: 0.9559122323989868.
Starting epoch 6.
	Training accuracy at step 500: 0.9559140801429749.
Starting epoch 7.
	Training accuracy at step 600: 0.9559153318405151.
Starting epoch 8.
Starting epoch 9.
	Training accuracy at step 700: 0.9559339284896851.
Starting epoch 10.
	Training accuracy at step 800: 0.9559292793273926.
Starting epoch 11.
	Training accuracy at step 900: 0.9559669494628906.
Starting epoch 12.
	Training accuracy at step 1000: 0.9559747576713562.
Starting epoch 13.
Starting epoch 14.
	Training accuracy at step 1100: 0.9559968709945679.
Starting epoch 15.
	Training accuracy at step 1200: 0.9561719298362732.
Starting epoch 16.
	Training accuracy at step 1300: 0.9564321041107178.
Startin

	Training accuracy at step 10600: 0.9577951431274414.
Starting epoch 138.
	Training accuracy at step 10700: 0.9578129649162292.
Starting epoch 139.
Starting epoch 140.
	Training accuracy at step 10800: 0.957796573638916.
Starting epoch 141.
	Training accuracy at step 10900: 0.9577940702438354.
Starting epoch 142.
	Training accuracy at step 11000: 0.9578118920326233.
Starting epoch 143.
Starting epoch 144.
	Training accuracy at step 11100: 0.9578264951705933.
Starting epoch 145.
	Training accuracy at step 11200: 0.9578276872634888.
Starting epoch 146.
	Training accuracy at step 11300: 0.9578378200531006.
Starting epoch 147.
Starting epoch 148.
	Training accuracy at step 11400: 0.9578518271446228.
Starting epoch 149.
	Training accuracy at step 11500: 0.9578602313995361.
Starting epoch 150.
	Training accuracy at step 11600: 0.957869827747345.
Starting epoch 151.
	Training accuracy at step 11700: 0.957878589630127.
Starting epoch 152.
Starting epoch 153.
	Training accuracy at step 11800: 0

KeyboardInterrupt: 

In [13]:
# Run predictions against the full test set.

predicted = sess.run([predictions], feed_dict={X: test_images28, training_mode: False})[0]
print(predicted)
print(len(predicted))
# # Calculate correct matches
# match_count = sum([int(y == y_) for y, y_ in zip(test_labels, predicted)])

# # Calculate the accuracy
# accuracy = match_count / len(test_labels)

# # Print the accuracy
# print("Accuracy: {:.3f}".format(accuracy))

[0 0 0 ... 0 0 0]
3300


In [14]:
np.sum(predicted)

84

In [15]:
def precision(labels, predicted):
    FP=0
    TP=0
    for i in range(len(labels)):
        if labels[i]=='1' and predicted[i]==1:
            TP+=1
        if labels[i]=='0' and predicted[i]==1:
            FP+=1
    print(TP)
    print(FP)
    return TP/(TP+FP)

#         scipy.misc.imsave("/Users/chenlingna/Desktop/output/"+str(i)+".JPG", test_images[i])

In [16]:
def recall(labels, predicted):
    FN=0
    TP=0
    for i in range(len(labels)):
        if labels[i]=='1' and predicted[i]==1:
            TP+=1
        if labels[i]=='1' and predicted[i]==0:
            FN+=1
    print("FN:")
    print(FN)
    return TP/(TP+FN)

In [17]:
# Calculate correct matches
match_count = sum([int(int(y) == y_) for y, y_ in zip(test_labels, predicted)])
print(match_count)
# Calculate the accuracy
accuracy = match_count / len(test_labels)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))
# Print the recall and precision
print("Recall: {:3f}".format(recall(test_labels, predicted)))
print("Precision: {:3f}".format(precision(test_labels, predicted)))

3134
Accuracy: 0.950
FN:
113
Recall: 0.215278
31
53
Precision: 0.369048
